# TNT Multi-Species: Flow Curve Fitting
> **Handbook:** This notebook demonstrates the TNT Multi-Species model. For complete mathematical derivations and theoretical background, see [TNT Multi-Species Documentation](../../docs/source/models/tnt/tnt_multi_species.rst).


**Estimated Time:** 3-5 minutes

## Welcome to TNT Multi-Species (Multiple Bond Types)

This notebook is the **gateway** to the TNT Multi-Species model — a generalization for networks with **N independent bond types**, each with its own modulus $G_i$ and lifetime $\tau_i$.

### What are Multi-Species Networks?

Materials with **heterogeneous crosslinks**:
- **Mixed metal-ligand coordination**: Cu²⁺ (fast) + Zn²⁺ (slow) bonds
- **Dual hydrogen bonding**: Weak H-bonds + strong ionic pairs
- **Hierarchical associations**: Multiple sticker types with different activation energies

Key feature: Each species evolves **independently** with its own conformation tensor $\mathbf{S}_i$, and the **total stress is the sum**:

$$
\boldsymbol{\tau} = \sum_{i=1}^N G_i \mathbf{S}_i
$$

### Multi-Species Signatures

| Feature | Prediction | Physical Origin |
|---------|------------|-----------------|
| **Flow curve** | Staged shear thinning | Weak bonds break first, then strong bonds |
| **SAOS** | Multiple crossovers, overlapping semicircles | Discrete relaxation spectrum |
| **Relaxation** | Multi-exponential decay | Each bond type contributes a mode |
| **Startup** | Multiple overshoots possible | Different bond breakage timescales |

### Multi-Species vs. Other TNT Models

| Model | # Species | Species Coupling | Relaxation Spectrum |
|-------|-----------|------------------|---------------------|
| **SingleMode** | 1 | N/A | Single Debye |
| **Cates** | 1 (effective) | Reptation + scission | Single $\tau_d$ (geometric mean) |
| **Loop-Bridge** | 2 | Loop↔bridge conversion | Two modes (coupled) |
| **Multi-Species** | N | **Independent** | N modes (uncoupled) |

> **Multi-Species Handbook**  
> For N-species framework, staged relaxation, and parameter extraction, see:  
> [../../docs/source/models/tnt/tnt_multi_species.rst](../../docs/source/models/tnt/tnt_multi_species.rst)

---

## Learning Objectives

1. Understand multi-species network physics
2. Fit TNT Multi-Species model to flow curve data
3. Extract individual species moduli and lifetimes
4. Predict staged shear thinning
5. Analyze bond hierarchy from parameter values
6. Perform Bayesian inference

## Prerequisites

- TNT fundamentals (NB01)
- Multi-mode relaxation concepts

## Runtime Estimate

- NLSQ: ~10-20s (N-species ODE)
- Bayesian: ~120-240s
- Total: ~10-15 min

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTMultiSpecies

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_multi_species_param_names,
    plot_multi_species_spectrum,
    plot_mode_decomposition,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Multiple Independent Bond Species

The TNT multi-species model describes materials with multiple independent bond populations, each with its own modulus $G_i$ and lifetime $\tau_{b,i}$.

**Total stress:**
$$\sigma = \sum_{i=0}^{N-1} \sigma_i + \eta_s \dot{\gamma}$$

**Per-species contribution (steady shear):**
$$\sigma_i = \frac{G_i \tau_{b,i} \dot{\gamma}}{1 + (\tau_{b,i} \dot{\gamma})^2}$$

**Key physics:**
- Fast bonds (short $\tau_{b,0}$): Dominate high shear rates
- Slow bonds (long $\tau_{b,1}$): Dominate low shear rates
- Discrete relaxation spectrum with 2 modes
- Stress superposition from independent populations

**Parameters (2 species):**
- $G_0$: Fast species modulus [Pa]
- $\tau_{b,0}$: Fast species bond lifetime [s]
- $G_1$: Slow species modulus [Pa]
- $\tau_{b,1}$: Slow species bond lifetime [s]
- $\eta_s$: Solvent viscosity [Pa·s]

## Load Data

In [ ]:
gamma_dot, sigma = load_ml_ikh_flow_curve("ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.4f} to {gamma_dot.max():.2f} s^-1")
print(f"Stress range: {sigma.min():.2f} to {sigma.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', markersize=8)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ [s$^{-1}$]', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ [Pa]', fontsize=12)
ax.set_title('Carbopol Flow Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTMultiSpecies(n_species=2)
param_names = get_tnt_multi_species_param_names(n_species=2)
print(f"Parameters: {param_names}")

start_time = time.time()
model.fit(gamma_dot, sigma, test_mode="flow_curve", method="scipy")
nlsq_time = time.time() - start_time

print(f"\nNLSQ fit completed in {nlsq_time:.2f} s")
print("\nFitted parameters:")
for name in param_names:
    print(f"  {name}: {model.parameters.get_value(name):.6e}")

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(sigma, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, sigma, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Physical Analysis: Per-Species Stress Decomposition

In [ ]:
# Create prediction grid
gamma_dot_pred = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)

fig = plot_mode_decomposition(model, gamma_dot_pred, "flow_curve")
plt.close("all")
plt.close('all')

print("\nSpecies characteristics:")
G_0 = model.parameters.get_value('G_0')
tau_b_0 = model.parameters.get_value('tau_b_0')
G_1 = model.parameters.get_value('G_1')
tau_b_1 = model.parameters.get_value('tau_b_1')

print(f"\nSpecies 0 (fast):")
print(f"  G_0 = {G_0:.3e} Pa")
print(f"  tau_b_0 = {tau_b_0:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_0 = {1/tau_b_0:.3e} s^-1")

print(f"\nSpecies 1 (slow):")
print(f"  G_1 = {G_1:.3e} Pa")
print(f"  tau_b_1 = {tau_b_1:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_1 = {1/tau_b_1:.3e} s^-1")

print(f"\nLifetime ratio: tau_b_1/tau_b_0 = {tau_b_1/tau_b_0:.2f}")
print(f"Modulus ratio: G_1/G_0 = {G_1/G_0:.2f}")

## Discrete Relaxation Spectrum

In [ ]:
fig = plot_multi_species_spectrum(model)
plt.close("all")
plt.close('all')

print("\nSpectrum interpretation:")
print("- Two discrete relaxation times from two bond species")
print("- Fast species contributes at short times/high frequencies")
print("- Slow species contributes at long times/low frequencies")
print("- No continuous distribution (discrete modes only)")

## Important Note on Model Applicability

**Carbopol is not a multi-species network.** It is a single cross-linked polymer gel with a continuous relaxation spectrum. This notebook demonstrates:
1. The multi-species workflow
2. Species decomposition analysis tools
3. How to interpret discrete spectra

Multi-species models are appropriate for:
- Binary blends (two polymer types)
- Multi-length-scale networks
- Materials with distinct fast/slow relaxation mechanisms

For Carbopol, single-species TNT models are more physically appropriate.

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        gamma_dot, sigma,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(result_bayes, 'to_inference_data'):
    display_arviz_diagnostics(result_bayes, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## NLSQ vs Bayesian Parameter Comparison

In [ ]:
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)

## Posterior Predictive: Flow Curve

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(result_bayes, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot, sigma, model, result_bayes,
        test_mode="flow_curve", param_names=param_names,
        log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation

**Multi-species stress contributions:**
- Fast species dominates at high shear rates (short timescale)
- Slow species dominates at low shear rates (long timescale)
- Crossover region shows mixed contributions

**Discrete spectrum:**
- Two well-separated relaxation times
- No intermediate modes (discrete, not continuous)
- Lifetime ratio indicates timescale separation

**Uncertainty quantification:**
- Bayesian posterior captures parameter correlations
- Predictive uncertainty from finite data
- Species parameters may trade off in fitting

## Save Results

In [ ]:
save_tnt_results(model, result_bayes, "multi_species", "flow_curve", param_names)
print("Results saved successfully.")

## Key Takeaways

1. **Multi-species framework**: Independent bond populations with different lifetimes
2. **Stress superposition**: Total stress is sum of species contributions
3. **Timescale separation**: Fast/slow species dominate different shear rate regimes
4. **Discrete spectrum**: Finite number of relaxation modes
5. **Workflow demonstration**: This notebook shows analysis tools, not physical appropriateness for Carbopol
6. **Bayesian inference**: Quantifies parameter uncertainty and correlations
7. **Species decomposition**: Visualization tools reveal individual contributions

## Next Steps

- **Notebook 20**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
